In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker import Session

sagemaker_session = Session()
bucket = 'path.to.dataset.bucket'
role =  sagemaker.get_execution_role()

instance_type = 'ml.m5.2xlarge'
epochs = 20


In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig, DebuggerHookConfig, CollectionConfig

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path='s3://{}/tensorboard'.format(bucket),
    container_local_output_path='/opt/ml/output/tensorboard'
)

pytorch_estimator = PyTorch('train.py',
                            source_dir='./src',
                            sagemaker_session=sagemaker_session,
                            role=role,
                            instance_type=instance_type,
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            tensorboard_output_config=tensorboard_output_config,
                            hyperparameters = {'epochs': epochs, 'batch-size': 64, 'learning-rate': 0.1})

pytorch_estimator.fit({'train': 's3://{}/dataset2'.format(bucket),
                       'test': 's3://{}/dataset2'.format(bucket)})

In [ ]:
tensorboard_output_path = pytorch_estimator.latest_job_tensorboard_artifacts_path()
print(tensorboard_output_path)

